In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

---
#New York CitiBikes 
#The Prediction of Speed and the Influence of Terrain and Weather
---

---
#2. Integrating with other data
---

---
##Motivation - Adds a twist to the project
---

---
#2. a. Accessing the MapQuest API
---

---
##MapQuest have recently revamped their service offerings - so the map quest key below no longer works
---

In [2]:
data = pd.read_csv("/Users/mrgholt/GADS-22-NYC/Citibike_Data/raw/2014-01 - Citi Bike trip data.csv")
dfN = data
dfN.starttime = pd.to_datetime(data.starttime)
dfN.stoptime = pd.to_datetime(data.stoptime)

In [4]:
dfN.columns

Index([u'tripduration', u'starttime', u'stoptime', u'start station id', u'start station name', u'start station latitude', u'start station longitude', u'end station id', u'end station name', u'end station latitude', u'end station longitude', u'bikeid', u'usertype', u'birth year', u'gender'], dtype='object')

In [5]:
from_lat = dfN.iloc[0]['start station latitude']
print from_lat
from_long = dfN.iloc[0]['start station longitude']
print from_long
to_lat = dfN.iloc[0]['end station latitude']
print to_lat
to_long = dfN.iloc[0]['end station longitude']
print to_long

40.71117444
-73.99682619
40.71729
-73.996375


In [6]:
str_a = "http://open.mapquestapi.com/directions/v2/route?key={:s}".format(myMapQuestKey)
str_b = "&from={:f},{:f}".format(from_lat, from_long)
str_c = "&to={:f},{:f}".format(to_lat,to_long)
str_d = "&routeType=bicycle&doReverseGeocode=True&narrativeType=none&cyclinggRoadFactor=1.0"

In [7]:
urlA = str_a + ',' + str_b + ',' + str_c + ',' + str_d
print type(urlA)
print urlA

<type 'str'>
http://open.mapquestapi.com/directions/v2/route?key=Fmjtd%7Cluurnua2nq%2Ca2%3Do5-9w859r,&from=40.711174,-73.996826,&to=40.717290,-73.996375,&routeType=bicycle&doReverseGeocode=True&narrativeType=none&cyclinggRoadFactor=1.0


In [8]:
import urllib

sock = urllib.urlopen(urlA)
content = sock.read() 
sock.close()

In [9]:
print content

The AppKey submitted with this request is invalid.


---
###The distance was available via 'content'
---

---
##Elevation
---

In [10]:
str_a = "http://open.mapquestapi.com/elevation/v1/profile?key={:s}".format(myMapQuestKey)
str_b = "&latLngCollection={:f},{:f},{:f},{:f}".format(from_lat,from_long,to_lat,to_long)

In [11]:
urlA2 = str_a + ',' + str_b
print type(urlA2)
print urlA2

<type 'str'>
http://open.mapquestapi.com/elevation/v1/profile?key=Fmjtd%7Cluurnua2nq%2Ca2%3Do5-9w859r,&latLngCollection=40.711174,-73.996826,40.717290,-73.996375


In [12]:
import urllib

sock = urllib.urlopen(urlA)
content = sock.read() 
sock.close()

In [13]:
print content

The AppKey submitted with this request is invalid.


---
###This provided the elevation at the starting point and ending point
###So any difference in elevation was the amount that the trip went up/down hill
###Note a terribly useful metric for NYC - wish I had done this with SF data!
---

---
#2. b. Accessing the Wunderground API
---

---
###Get the weather conditions at the starting station at the closest time to when the rider left the station
###We assumed that such conditions prevailed throughout the journey
###It's a pretty loose assumption because, depending on the route, wind could be a significant factor in an E-W or a N-S facing street
---

In [15]:
ts = pd.to_datetime(dfN.starttime.values[4563])
print ts

2014-01-01 17:51:08


In [16]:
ts.strftime('%Y%m%d')

'20140101'

In [17]:
hour = ts.strftime('%-H')
print hour
minute = ts.strftime('%M')
print minute

17
51


---
###The start station state was, obviously, NY
###I got the start station zip and city name from the MapQuest route
---

In [18]:
theDate = ts.strftime('%Y%m%d')

In [19]:
str_a = "http://api.wunderground.com/api/{:s}".format(myWunderGroundAPI)
str_b = "/history_" + theDate + "/q/" +  "NewYork" + "/" + "NewYork" + ".json"
urlB = str_a + str_b
print urlB

http://api.wunderground.com/api/34fd63423ce8c852/history_20140101/q/NewYork/NewYork.json


In [20]:
import urllib

sock = urllib.urlopen(urlB)
content = sock.read() 
sock.close()

In [21]:
print content


{
  "response": {
  "version":"0.1",
  "termsofService":"http://www.wunderground.com/weather/api/d/terms.html",
  "features": {
  "history": 1
  }
	}
		,
	"history": {
		"date": {
		"pretty": "January 1, 2014",
		"year": "2014",
		"mon": "01",
		"mday": "01",
		"hour": "12",
		"min": "00",
		"tzname": "America/New_York"
		},
		"utcdate": {
		"pretty": "January 1, 2014",
		"year": "2014",
		"mon": "01",
		"mday": "01",
		"hour": "17",
		"min": "00",
		"tzname": "UTC"
		},
		"observations": [
		{
		"date": {
		"pretty": "12:51 AM EST on January 01, 2014",
		"year": "2014",
		"mon": "01",
		"mday": "01",
		"hour": "00",
		"min": "51",
		"tzname": "America/New_York"
		},
		"utcdate": {
		"pretty": "5:51 AM GMT on January 01, 2014",
		"year": "2014",
		"mon": "01",
		"mday": "01",
		"hour": "05",
		"min": "51",
		"tzname": "UTC"
		},
		"tempm":"-3.9", "tempi":"25.0","dewptm":"-14.4", "dewpti":"6.1","hum":"45","wspdm":"5.6", "wspdi":"3.5","wgustm":"-9999.0", "wgusti":"-9999.0","wdird":"260"

In [22]:
import json
wdict = json.loads(content)

In [23]:
wdict

{u'history': {u'dailysummary': [{u'coolingdegreedays': u'0',
    u'coolingdegreedaysnormal': u'0',
    u'date': {u'hour': u'12',
     u'mday': u'01',
     u'min': u'00',
     u'mon': u'01',
     u'pretty': u'12:00 PM EST on January 01, 2014',
     u'tzname': u'America/New_York',
     u'year': u'2014'},
    u'fog': u'0',
    u'gdegreedays': u'0',
    u'hail': u'0',
    u'heatingdegreedays': u'36',
    u'heatingdegreedaysnormal': u'32',
    u'humidity': u'',
    u'maxdewpti': u'19',
    u'maxdewptm': u'-7',
    u'maxhumidity': u'59',
    u'maxpressurei': u'30.40',
    u'maxpressurem': u'1029',
    u'maxtempi': u'33',
    u'maxtempm': u'1',
    u'maxvisi': u'10',
    u'maxvism': u'16',
    u'maxwspdi': u'14',
    u'maxwspdm': u'23',
    u'meandewpti': u'13',
    u'meandewptm': u'-11',
    u'meanpressurei': u'30.34',
    u'meanpressurem': u'1027',
    u'meantempi': u'29',
    u'meantempm': u'-2',
    u'meanvisi': u'10',
    u'meanvism': u'16',
    u'meanwdird': u'314',
    u'meanwdire': u'

In [24]:
wdict['history'].keys()

[u'date', u'utcdate', u'observations', u'dailysummary']

In [25]:
print hour

17


In [26]:
print minute

51


In [27]:
wdict['history']['observations'][16]

{u'conds': u'Clear',
 u'date': {u'hour': u'17',
  u'mday': u'01',
  u'min': u'51',
  u'mon': u'01',
  u'pretty': u'5:51 PM EST on January 01, 2014',
  u'tzname': u'America/New_York',
  u'year': u'2014'},
 u'dewpti': u'15.1',
 u'dewptm': u'-9.4',
 u'fog': u'0',
 u'hail': u'0',
 u'heatindexi': u'-9999',
 u'heatindexm': u'-9999',
 u'hum': u'48',
 u'icon': u'clear',
 u'metar': u'METAR KNYC 012251Z AUTO 00000KT 10SM CLR 01/M09 A3034 RMK AO2 SLP267 T00061094',
 u'precipi': u'-9999.00',
 u'precipm': u'-9999.00',
 u'pressurei': u'30.32',
 u'pressurem': u'1026.7',
 u'rain': u'0',
 u'snow': u'0',
 u'tempi': u'33.1',
 u'tempm': u'0.6',
 u'thunder': u'0',
 u'tornado': u'0',
 u'utcdate': {u'hour': u'22',
  u'mday': u'01',
  u'min': u'51',
  u'mon': u'01',
  u'pretty': u'10:51 PM GMT on January 01, 2014',
  u'tzname': u'UTC',
  u'year': u'2014'},
 u'visi': u'10.0',
 u'vism': u'16.1',
 u'wdird': u'0',
 u'wdire': u'North',
 u'wgusti': u'-9999.0',
 u'wgustm': u'-9999.0',
 u'windchilli': u'-999',
 u'win

In [28]:
wdict['history']['observations'][16].keys()

[u'heatindexm',
 u'windchillm',
 u'wdire',
 u'wdird',
 u'windchilli',
 u'hail',
 u'heatindexi',
 u'precipi',
 u'thunder',
 u'pressurei',
 u'snow',
 u'pressurem',
 u'fog',
 u'icon',
 u'precipm',
 u'conds',
 u'tornado',
 u'hum',
 u'tempi',
 u'tempm',
 u'dewptm',
 u'rain',
 u'dewpti',
 u'date',
 u'visi',
 u'vism',
 u'utcdate',
 u'wgusti',
 u'metar',
 u'wgustm',
 u'wspdi',
 u'wspdm']

In [29]:
print "dew point in fahrenheit", wdict['history']['observations'][0]['dewpti']
print "precipitation in inches", wdict['history']['observations'][0]['precipi']
print "humidity as a percentage", wdict['history']['observations'][0]['hum']
print "snow", wdict['history']['observations'][0]['snow']
print "temperature in fahrenheit", wdict['history']['observations'][0]['tempi']
print "windspeed in mph", wdict['history']['observations'][0]['wspdi']
print "visibility in miles", wdict['history']['observations'][0]['visi']
print "windchill in fahrenheit", wdict['history']['observations'][0]['windchilli']
print "heat index in fahrenheit", wdict['history']['observations'][0]['heatindexi']
print "fog", wdict['history']['observations'][0]['fog']
print "rain", wdict['history']['observations'][0]['rain']
print "hail", wdict['history']['observations'][0]['hail']
print "thunder", wdict['history']['observations'][0]['thunder']
print "tornado", wdict['history']['observations'][0]['tornado']

dew point in fahrenheit 6.1
precipitation in inches -9999.00
humidity as a percentage 45
snow 0
temperature in fahrenheit 25.0
windspeed in mph 3.5
visibility in miles 10.0
windchill in fahrenheit 20.7
heat index in fahrenheit -9999
fog 0
rain 0
hail 0
thunder 0
tornado 0


---
#Let's read in a year's worth of processed data
---

In [30]:
location = "/Users/mrgholt/GADS-22-NYC/Citibike_Data/"
dfJanComplete = pd.read_csv(location + "dfJanComplete.csv")
dfFebComplete = pd.read_csv(location + "dfFebComplete.csv")
dfMarComplete = pd.read_csv(location + "dfMarComplete.csv")
dfAprComplete = pd.read_csv(location + "dfAprComplete.csv")
dfMayComplete = pd.read_csv(location + "dfMayComplete.csv")
dfJunComplete = pd.read_csv(location + "dfJunComplete.csv")
dfJulComplete = pd.read_csv(location + "dfJulComplete.csv")
dfAugComplete = pd.read_csv(location + "dfAugComplete.csv")
dfSepComplete = pd.read_csv(location + "dfSepComplete.csv")
dfOctComplete = pd.read_csv(location + "dfOctComplete.csv")
dfNovComplete = pd.read_csv(location + "dfNovComplete.csv")
dfDecComplete = pd.read_csv(location + "dfDecComplete.csv")

In [31]:
dfDecComplete = dfDecComplete.drop(1000)

In [32]:
for i in dfDecComplete.columns:
    print i

Unnamed: 0
day
tripduration
starttime
stoptime
start.station.id
start.station.name
start.station.latitude
start.station.longitude
end.station.id
end.station.name
end.station.latitude
end.station.longitude
bikeid
usertype
birth.year
gender
distance
zip
elev
dew
precip
hum
snow
temp
windspeed
visibility
windchill
heatindex
fog
rain
hail
thunder
tornado


In [42]:
dfDisplay = dfDecComplete[['start.station.id', 'end.station.id', 'distance', 'elev', 'dew', \
                           'precip', 'hum', 'snow', 'temp', 'windspeed', 'visibility', 'windchill', 'heatindex',\
                           'fog', 'rain', 'hail', 'thunder', 'tornado']]
dfDisplay.columns = ['start', 'end', 'distance', 'elev', 'dew', \
                           'precip', 'hum', 'snow', 'temp', 'windspeed', 'visibility', 'windchill', 'heatindex',\
                           'fog', 'rain', 'hail', 'thunder', 'tornado']
dfDisplay[0:25]

,start,end,distance,elev,dew,precip,hum,snow,temp,windspeed,visibility,windchill,heatindex,fog,rain,hail,thunder,tornado
0,318,477,0.769,32791,28.9,-9999.00,41,0,52.0,5.8,10,-999.0,-9999,0,0,0,0,0
1,359,477,1.122,32791,41.0,0.02,86,0,45.0,6.9,7,41.2,-9999,0,1,0,0,0
2,458,521,0.811,0,41.0,0.02,86,0,45.0,6.9,7,41.2,-9999,0,1,0,0,0
3,417,415,0.873,0,19.9,-9999.00,61,0,32.0,0.0,10,-999.0,-9999,0,0,0,0,0
4,435,509,0.661,-12,17.1,-9999.00,41,0,39.0,9.2,10,32.8,-9999,0,0,0,0,0
5,472,528,0.366,-4,3.9,-9999.00,37,0,27.0,-9999.0,10,-999.0,-9999,0,0,0,0,0
6,525,490,0.865,21,28.9,-9999.00,82,0,34.0,-9999.0,6,-999.0,-9999,0,0,0,0,0
7,2012,379,1.203,10,37.9,-9999.00,59,0,52.0,3.5,5,-999.0,-9999,0,0,0,0,0
8,2012,379,1.203,10,35.1,-9999.00,89,0,37.9,3.5,3,35.5,-9999,0,0,0,0,0
9,329,147,0.351,8,43.0,0.02,86,0,46.9,4.6,10,-999.0,-9999,0,0,0,0,0
